
# Hypothesis Testing with Neuro Data - RM ANOVA

## NOTES

- This notebook is intended to walk through preparing my binge drinking data for Hypothesis Testing
- Specifically, in this notebook I will attempt to use the most appropriate stat tests, which are not taught in the Learn curriculum


- **Repeated Measures ANOVA in Python**
-  **Two way RM ANOVA**

### REFERENCES

- [RM ANOVA IN Python with Statsmodels](https://www.marsja.se/repeated-measures-anova-in-python-using-statsmodels/)
- One-way RM ANOVA (other packages): https://www.marsja.se/repeated-measures-anova-using-python/
- Two-Way: https://marsja.se/two-way-anova-repeated-measures-using-python/


# Statistical Tests Summary Table



| Parametric tests (means) | Function | Nonparametric tests (medians) | Function |
 | --- | --- | --- | --- |
 | 1-sample t test |`scipy.stats.ttest_1samp()`|  1-sample Wilcoxon |`scipy.stats.wilcoxon`|
 | 2-sample t test |`scipy.stats.ttest_ind()` | Mann-Whitney U test |`scipy.stats.mannwhitneyu()` |
 | One-Way ANOVA | `scipy.stats.f_oneway()` | Kruskal-Wallis | `scipy.stats.kruskal` | 
 
 
 | Factorial DOE with one factor and one blocking variable |Friedman test  |


# Real-World Science / Experimental Design

- [James' Neuroscience Research Poster: Society for Neuroscience 2016](https://drive.google.com/open?id=14z2dUdPB_8ei3HA7R1j3ylwEP0kVZhJq)

### The Role of Stress Neurons in the Amygdala in Addiction/Binge Drinking

- We will be talking through some of the experiments from my Postdoctoral research on the roll of stress neurons in the escalation of binge drinking.


#### The Opponent-Process Theory of Addiction 


<img src="https://raw.githubusercontent.com/jirvingphd/fsds_pt_100719_cohort_notes/master/Images/robinson-berridge-fig1.jpg">

## Hypothesis 

- Based on prior evidence in the field, stress neurons in the amygdala are believed to be responsible for the negative emotions that promote binge consumption to relieve negative symptoms

$ H_1$: Increasing the activity of stress neurons (CRF neurons) in the amygdala will increase the amount of alcohol consumed by binge-drinking mice.

$H_0$: Stimulation of CRF neurons has no effect on the amount of alcohol consumed.

<img src="https://raw.githubusercontent.com/jirvingphd/fsds_pt_100719_cohort_notes/master/Images/jmi_poster_preds1.png" width=60%>

## Experimental Design

<img src="https://raw.githubusercontent.com/jirvingphd/fsds_pt_100719_cohort_notes/master/Images/opto_6steps.jpg">

<img src="https://raw.githubusercontent.com/jirvingphd/hypothesis_testing_lessons/master/images/jmi_poster_fig1_no_mouse.png">

<!---
<img src="https://raw.githubusercontent.com/jirvingphd/hypothesis_testing_lessons/master/images/jmi_poster_fig1.png">--->

<img src="https://raw.githubusercontent.com/jirvingphd/fsds_pt_100719_cohort_notes/master/Images/jmi_poster_fig2.png">

<!---
<img src="https://raw.githubusercontent.com/jirvingphd/fsds_100719_cohort_notes/master/images/sect_20_neuro_data.png">')
--->

## Hypothesis Testing: Mouse Data

### Hypothesis
> Question: does stimulation of CRF Neurons in the central amygdala increase alcohol consumption?

- Metric: licks for alcohol
- Two groups: Control and Experimental (ChR2)


- $H_1$: There is a significant difference in average licks for alcohol between control and experimental/stimulated mice.

- $H_0$: There is no significant difference in licks for alcohol between control and experimental/stimulated mice.

$\alpha$=0.05


### Step 1: which type of test?

- What type of data?
    - Numerical
- How many groups?
    - 2 groups

In [ ]:
# from IPython.display import HTML
# HTML('<img src="https://raw.githubusercontent.com/jirvingphd/fsds_100719_cohort_notes/master/images/sect_20_neuro_data.png">')

In [ ]:
!pip install -U fsds
from fsds.imports import *

In [ ]:
plt.style.use('seaborn-poster')
pd.set_option('display.max_columns',0)
pd.set_option('display.precision',3)

# Obtaining/Preprocessing Data

In [ ]:
df = pd.read_csv("../Neuroscience/mouse_drinking_data_cleaned.csv",index_col=0)
df.drop('Sex',inplace=True,axis=1)
df 

#### Laying Out Our Approach

1. Make a **dict/lists of the column names** that should be **averaged together** (`col_dict`)

2. Make a new df of means using `col_dict`

3. Make a grp dict using  `df_means.groupby('Group').groups` 

- Visualize the two populations

- Prepare for hypothesis tests
    - Either use `grps` dict to reference the correct columsn to pass into tests

<!---
**Variables:**

- `col_dict` (dict): dict of column names to be grouped together for means
- `df_means` (df): df of col_dict column means.
- `grps` (dict): groupby dict where keys = 'Group' column and values = row indices

- `data` (dict): Dictionary of...
    - Series of each phase by group? --->

In [ ]:
phases = ['BL','S','PS','R1','R2']
col_dict = {}
for phase in phases:
    col_dict[phase] = [col for col in df.columns if col.startswith(phase) ]
col_dict    

In [ ]:
# ## Make lists of columns to be averaged together
# BL_cols = [col for col in df.columns if 'BL' in col]
# PS_cols = [col for col in df.columns if 'PS' in col]
# R1_cols = [col for col in df.columns if 'R1_' in col]
# R2_cols = [col for col in df.columns if 'R2_' in col]
# S_cols= [col for col in df.drop(PS_cols,axis=1) if 'S' in col]

# col_dict = {'BL':BL_cols,
#            'S':S_cols,
#            'PS':PS_cols,
#            'R1':R1_cols,
#             'R2':R2_cols}


# col_dict

In [ ]:
# phase_dict = {col:phase for phase,col}

In [ ]:
list(map(lambda x: col_dict[x],col_dict))

In [ ]:
phase_dict = {}
for phase,colnames in col_dict.items():
    for col in colnames:
        phase_dict[col] = phase


phase_dict

In [ ]:
df

### Melting DF

In [ ]:
## MELTING THE DATA INTO FORM WORKABLE FOR PLOTS/ANALYSIS
df2= pd.melt(df.reset_index(),id_vars=['Mouse_ID','Group'],
              value_name='Licks',var_name='Day')
df2

In [ ]:
## Mapping Phase from Phase Dict
df2['Phase'] = df2['Day'].map(phase_dict)

## Getting Day of Phase
df2['Day_of_Phase'] = df2['Day'].apply(lambda x: x[-1])
# df2 = df2.dropna(subset=['Phase'])
df2

In [ ]:
# df2.to_csv('../Neuroscience/mouse_drinking_data_melted.csv',index=False)

In [ ]:
# pd.read_csv('../Neuroscience/mouse_drinking_data_melted.csv')

# 📕 TO DO (06/30/20): RM_ANOVA

> ## Main Resources
- https://www.marsja.se/repeated-measures-anova-in-python-using-statsmodels/

> ### Additional Resources
- https://www.marsja.se/three-ways-to-carry-out-2-way-anova-with-python/
- https://www.marsja.se/four-ways-to-conduct-one-way-anovas-using-python/

In [ ]:
df_melt = pd.read_csv('../Neuroscience/mouse_drinking_data_melted.csv')
df_melt

In [ ]:
del df2
# del df_melt

In [ ]:
df3 = df.pivot_table(values='Licks',index=['Group','Mouse_ID'],
                      columns=['Phase','Day_of_Phase'])
df3

In [ ]:
# df.groupby(['Group','Mouse_ID'])['Mouse_ID'].count()#.max()

### NOTES (06/29/30)

- There are 52 observations for ChR2, but only 36 for Control.
    - This is probably why RMANOVA returns an error
    
- Possible Solutions:
    - Remove mice from ChR2 group to match Control
    - Resample from Control Group to match ChR2

In [ ]:
data = {}
for grp in df.groupby('Group').groups:
    data[grp] = df.groupby('Group').get_group(grp)
    print(grp)
    print(len(data[grp]['Mouse_ID'].unique()))
    print()

In [ ]:
n_to_match = 13

### Identifying which Mice to Remove (outliers)

In [ ]:
# ## Making df that can be used with plotly express
# plotly_df = df2.copy()
# # plotly_df['Day_of_Phase'] = plotly_df['Day'].apply(lambda x: x[-1])
# plotly_df

In [ ]:
import plotly.express as px
px.box(df,x='Phase',y='Licks',
       color='Group',hover_name='Mouse_ID',height=400)#,
#       category_orders=['BL','S','PS','R1','R2'])#,points='suspectedoutliers')

> - Mice to Remove (try normalized first):
    - ChR2:
        - 20
    - Control:
        - 

# STOPPING POINT [06/30/20]

In [ ]:
df_anova_phases = df.pivot_table(index=['Group','Mouse_ID'],values='Licks',columns=['Phase'])
display(df_anova_phases)

In [ ]:
df

### Equalizing N's|

In [ ]:
n_chr2 = len(df_anova_phases.loc['ChR2'])
n_control= len(df_anova_phases.loc['Control'])
n_to_match = max( n_chr2,n_control)

In [ ]:
df

In [ ]:
## Resample dfa


df_list = []
# df_anova_phases.reset_index(inplace=True)
for group in df["Group"].unique():
    grp_df = df.groupby('Group').get_group(group)
    grp_df = grp_df.sample(n=n_to_match,replace=True,random_state=123)
    df_list.append(grp_df)
df


In [ ]:
from statsmodels.stats.anova import AnovaRM
df_resamp = pd.concat(df_list)
df_resamp

In [ ]:
import plotly.express as px
px.box(df_resamp,x='Phase',y='Licks',
       color='Group',hover_name='Mouse_ID',height=400)#,
#       category_orders=['BL','S','PS','R1','R2'])#,points='suspectedoutliers')

In [ ]:
# n_to_match = df2_eq

In [ ]:
# grp_dict = df_anova_phases.groupby('Group').groups

# ## Match the largest number of subjects
# n_to_match = max(list(map(lambda x: len(x),grp_dict.values())))
# n_to_match

In [ ]:
# df_list = []
# for group in df_anova_phases["Group"].unique():
#     grp_df = df_anova_phases.groupby('Group').get_group(group)
#     grp_df = grp_df.sample(n=n_to_match,replace=True)
#     df_list.append(grp_df)

In [ ]:
# df_anova_equal = pd.concat(df_list)
# df_anova_equal

### RM ANOVA

In [ ]:
df

In [ ]:
# # df2.pivot_table(index=['Group','Mouse_ID'],columns=['Day'])
# df_resamp.reset_index(drop=True,inplace=True)#.pivot_table(index=['Group','Day_of_Phase','Mouse_ID'])#,columns=['Day'])
# df_resamp

In [ ]:
# df_resamp['Phase'].value_counts()

In [ ]:
aovrm = AnovaRM(df, 'Licks', 'Mouse_ID', within=['Phase','Group'],aggregate_func=np.nanmean).fit()
aovrm.summary()

In [ ]:
df2.isna().sum()

In [ ]:
import statsmodels
statsmodels.__version__

In [ ]:
anovarm = AnovaRM(df2,'Licks','Mouse_ID',within=['Day'],
                 aggregate_func='mean')#between=['Group'],
res = anovarm.fit()
print(res)

# OLD PREPROCESSING

In [ ]:
mean_to_df = {}
for k, cols in col_dict.items():
    mean_to_df[k] = df[cols].mean(axis=1)
    
df_means = pd.concat([df[['Mouse ID','Group','Sex']],
                      pd.DataFrame(mean_to_df)],axis=1)
df_means

In [ ]:
# df_means.to_csv('../datasets/neuro_mouse_dirnking.csv',index=False)
# df_means = pd.read_csv('../datasets/neuro_mouse_dirnking.csv')
# df_means

### Getting Group Data For EDA & Testing

In [ ]:
df_means.head()

In [ ]:
## Get grps 

## Two different ways of using groupby
grps = df_means.groupby('Group').groups

grp_control = df_means.loc[grps['Control']]
grp_exp = df_means.groupby('Group').get_group('ChR2')
#grpB
# grps
display( grp_exp.head().style.set_caption('Exp/ChR2'),
        grp_control.head().style.set_caption('Controls'))

In [ ]:
## Group counts
len(grp_control), len(grp_exp)

In [ ]:

# sns.distplot(grp_control['BL'])

In [ ]:
# sem(grp_exp['BL'])
# sem(grp_control['BL'])

In [ ]:
from scipy.stats import sem
## showing off SEM bars
f,ax = plt.subplots(figsize=(5,5))
ax.bar('Exp',grp_exp['BL'],yerr=sem(grp_exp['BL']))
ax.bar('Control',grp_control['BL'],yerr=sem(grp_control['BL']))

In [ ]:
def plot_dists(grp1,grp2,col='BL',name1='Exp',name2='Control'):

    ## Defining "gridspec_kws" for plt.subplots()
    ## This will make our first plot 3 times wider than the second.
    gs_kw = dict(width_ratios=[3, 1])
    
    fig, axes = plt.subplots(figsize=(10,4),ncols=2,
                             gridspec_kw=gs_kw,constrained_layout=True)

    ## Defining the data 
    group1 = {'name':name1, 
             'data':grp1[col],
             'plot_specs':{
                 'hist_kws':dict(color='b', lw=2,ls='-'),
                 'kde_kws':dict(color='b',lw=1,ls='-'),
                 'label':f"{name1} (n={len(grp1[col])})"}
             }
    
    group2 = {'name':name2, 
             'data':grp2[col],
              'plot_specs':{
                  'hist_kws':dict(color='orange', lw=2,ls='-'),
                  'kde_kws':dict(color='orange',lw=1,ls='-'),
                   'label':f"{name2} (n={len(grp2[col])})"}
             }
    
    
    ax=axes[0]
    sns.distplot(group1['data'], **group1['plot_specs'],ax=axes[0])
    sns.distplot(group2['data'], **group2['plot_specs'],ax=axes[0])
    ax.legend()
    
    ax.set(ylabel="Density")
    ax.set(xlabel='Number of Licks')
    
    
    ax = axes[1]
    ax.bar(group1['name'],group1['data'].mean(),
          yerr=sem(group1['data']))

    ax.bar(group2['name'],group2['data'].mean(),
          yerr=sem(group2['data']))    
    
    return fig,ax

In [ ]:
fig,ax = plot_dists(grp_control,grp_exp,col='R',name1='ChR2', name2='Control')

### Writing functions to test assumptions

In [ ]:
import scipy.stats as stats
stats.normaltest(grp_control['BL'])

In [ ]:
def test_normality(grp_control,col='BL',alpha=0.05):
    import scipy.stats as stats
    stat,p =stats.normaltest(grp_control[col])
    if p<alpha:
        print(f"Normal test p value of {np.round(p,3)} is < {alpha}, therefore data is NOT normal.")
    else:
        print(f"Normal test p value of {np.round(p,3)} is > {alpha}, therefore data IS normal.")
    return p

def test_equal_variance(grp1,grp2, alpha=.05):
    stat,p = stats.levene(grp1,grp2)
    if p<alpha:
        print(f"Levene's test p value of {np.round(p,3)} is < {alpha}, therefore groups do NOT have equal variance.")
    else:
        print(f"Normal test p value of {np.round(p,3)} is > {alpha},  therefore groups DOES have equal variance.")
    return p



# def test_assumptions(*args, normal=True,equal_var=True):
#     pass

In [ ]:
test_normality(grp_control,col='S'), test_normality(grp_exp,col='S');

In [ ]:
def Cohen_d(group1, group2):
    """
    Compute Cohen's d.
    
    Args:
        group1: Series or NumPy array
        group2: Series or NumPy array

    Returns:
        d (float): effect size statistic

    Interpretation:
    > Small effect = 0.2
    > Medium Effect = 0.5
    > Large Effect = 0.8
    """
    diff = group1.mean() - group2.mean()

    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()

    # Calculate the pooled threshold as shown earlier
    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    
    # Calculate Cohen's d statistic
    d = diff / np.sqrt(pooled_var)
    
    return d

## NOTE: Functions below pasted in due to time. Will construct smaller but similar functions together next class

In [ ]:
def plot_statplot(df_means,grps=None,
                  group_col='Group',data_col='BL'):
    
    if grps is None:
        grps = df_means.groupby(group_col).groups

    ## Examine KDEs for BL
    fig= plt.figure(figsize=(10,6))
    axes=['','']
    # Define gridspec to create grid coordinates             
    gs = fig.add_gridspec(nrows=1,ncols=9)
    axes[0] = fig.add_subplot(gs[0,0:7])
    axes[1] = fig.add_subplot(gs[0,7:])

    data1=df_means.loc[grps['ChR2'],data_col]
    data2=df_means.loc[grps['Control'],data_col]
    
    group1 = {'name':'ChR2',
             'data':data1,#df_means.loc[grps['ChR2'],data_col],
             'n':len(data1)}
    plot1 = {'hist_kws':dict(color='blue',lw=2, ls='-')}#,bins='auto')}

    group2 = {'name':'Control',
             'data':data2,#df_means.loc[grps['Control'],data_col],
             'n':len(data2)}
    plot2 = {'hist_kws':dict(color='orange',lw=2, ls='-')}#,bins='auto')}
    
    ax = axes[0]
    label1= f"{group1['name']} n={group1['n']}"
    sns.distplot(group1['data'], label=label1,
                 ax=ax, hist_kws=plot1['hist_kws'])
    # ax.legend()

    label2= f"{group2['name']} n={group2['n']}"
    sns.distplot(group2['data'], label=label2,
                 ax=ax,hist_kws=plot2['hist_kws'])
    ax.legend()

    

    ax.axvline(group1['data'].mean(),color=plot1['hist_kws']['color'], ls='--')
    ax.axvline(group2['data'].mean(),color=plot2['hist_kws']['color'], ls='--')


    ax = axes[1]

    ax.bar(group1['name'],group1['data'].mean(),
          yerr=sem(group1['data']))

    ax.bar(group2['name'],group2['data'].mean(),
          yerr=sem(group2['data']))
    
    plt.suptitle(f"Phase = {data_col}",fontsize=20)
    
    return fig, ax

In [ ]:
def test_assumptions(df_means,grps=None,
                     group_col='Group',
                     grp1='ChR2',
                     grp2='Control',
                     data_col='BL',
                    plot_data=False):
    """MASSIVE FUNCTION PASTED IN DUE TO VERY LATE STUDY GROUP
    WE WILL CONSTRUCT A BETTER/SIMPLER VERSION OF THIS TOGETHER IN NEXT STUDY GROUP."""
    
    if grps is None:
        grps = df_means.groupby(group_col).groups
        
        
    group1 = {'name':grp1,
              'data':df_means.loc[grps[grp1],data_col]}
    
    group2 = {'name':grp2,
              'data':df_means.loc[grps[grp2],data_col]}
    
    results = [['Col','Test','Group(s)','Stat','p','p<.05']]
    
    ## Normality testing
    stat,p = stats.normaltest(group1['data'])
    results.append([data_col,'Normality',group1['name'],
                  stat, p, p<.05])
    
    stat,p = stats.normaltest(group2['data'])    
    results.append([data_col,'Normality',group2['name'],
                  stat, p, p<.05])
    ## Homo. of Variance Testing
    stat,p = stats.levene(group1['data'],group2['data'])
    results.append([data_col,'Equal Variance','Both',
                  stat, p, p<.05])
    
    
    ## Parametric T-Test
    stat,p = stats.ttest_ind(group1['data'],group2['data'])
    results.append([data_col,'T-Test 2samp','Both',stat,p,p<.05])
    
    ## Non-Parametric MWU
    stat,p = stats.mannwhitneyu(group1['data'],group2['data'])
    results.append([data_col,'Mann Whitney U','Both',stat,p,p<.05])
    
    ## Effect size with Cohen's d
    d = Cohen_d(group1['data'],group2['data'])
    results.append([data_col, "Cohen's d", 'Both','','',d])
    
#     if plot_data:
#         plot_dists(grp, col=data_col)
    
    return pd.DataFrame(results[1:],columns=results[0])

res = test_assumptions(df_means)


In [ ]:
for phase in ['BL','S','PS','R']:
    print('---'*30)

    res = test_assumptions(df_means,data_col=phase)
    display(res)
    
    fig,ax = plot_statplot(df_means, data_col=phase)
    plt.show()

# fig,ax = plot_dists(grp_control,grp_exp,col='R',name1='ChR2 Mice', name2='Control Mice')


## CONCLUSION
- Running the correct test according to the assumptions of normality and equal variance will ensure you can get the correct test result.

- Notice how the last phase (R) did NOT come back as significant when we ran the t-test, but DID come back significant when we performed the Mann Whitney U instead. 



# APPENDIX

(https://www.statsmodels.org/stable/generated/statsmodels.stats.multicomp.pairwise_tukeyhsd.html)

# APPENDIX

## Effect Size Visual
- https://rpsychologist.com/d3/NHST/


## Ephys Figure
<img src="https://raw.githubusercontent.com/jirvingphd/fsds_pt_100719_cohort_notes/master/Images/jmi_fig1.png">

## Statistical Analysis Pipeline

1. **Test for Normality**
    - D'Agostino-Pearson's normality test<br>
    ```scipy.stats.normaltest```
    - Shapiro-Wilik Test<br>
    ```scipy.stats.shapiro```<br>
    
    
2. **Test for Homogeneity of Variance**

    - Levene's Test<br>
    ```scipy.stats.levene```


3. **Choose appropriate test based upon 1. and 2.** <br> 
    - T Test (1-sample)
        - `stats.ttest_1samp()`
    - T Test (2-sample)
        - `stats.ttest_ind()`
        - [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
    - Welch's T-Test (2-sample)
        - `stats.ttest_ind(equal_var=False)`
        - [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
        
    - Mann Whitney U
        - `stats.mannwhitneyu()`
        - [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html)
    - ANOVA 
        - `stats.f_oneway()`
        - [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html)
    - Tukey's
     - `statsmodels.stats.multicomp.pairwise_tukeyhsd`
     -[docs](https://www.statsmodels.org/stable/generated/statsmodels.stats.multicomp.pairwise_tukeyhsd.html)
    

4. **Calculate effect size for significant results.**
    - Effect size: [cohen's d](https://stackoverflow.com/questions/21532471/how-to-calculate-cohens-d-in-python)
    - Interpretation:
        - Small effect = 0.2 ( cannot be seen by naked eye)
        - Medium effect  = 0.5
        - Large Effect = 0.8 (can be seen by naked eye)
        
5. **If significant, follow up with post-hoc tests (if have more than 2 groups)**
    - [Tukey's](https://www.statsmodels.org/stable/generated/statsmodels.stats.multicomp.pairwise_tukeyhsd.html)


In [ ]:
# def test_assumptions(df_means,grps=None,
#                      group_col='Group',
#                      grp1='ChR2',
#                      grp2='Control',
#                      data_col='BL'):
    
#     if grps is None:
#         grps = df_means.groupby(group_col).groups
        
        
#     group1 = {'name':grp1,
#               'data':df_means.loc[grps[grp1],data_col]}
    
#     group2 = {'name':grp2,
#               'data':df_means.loc[grps[grp2],data_col]}
    
#     results = [['Col','Test','Group(s)','Stat','p','p<.05']]
    
#     ## Normality testing
#     stat,p = stats.normaltest(group1['data'])
#     results.append([data_col,'Normality',group1['name'],
#                   stat, p, p<.05])
    
#     stat,p = stats.normaltest(group2['data'])    
#     results.append([data_col,'Normality',group2['name'],
#                   stat, p, p<.05])
#     ## Homo. of Variance Testing
#     stat,p = stats.levene(group1['data'],group2['data'])
#     results.append([data_col,'Equal Variance','Both',
#                   stat, p, p<.05])
    
#     ## Parametric T-Test
#     stat,p = stats.ttest_ind(group1['data'],group2['data'])
#     results.append([data_col,'T-Test 2samp','Both',stat,p,p<.05])
    
#     ## Non-Parametric MWU
#     stat,p = stats.mannwhitneyu(group1['data'],group2['data'])
#     results.append([data_col,'Mann Whitney U','Both',stat,p,p<.05])
    
#     ## Effect size with Cohen's d
#     d = Cohen_d(group1['data'],group2['data'])
#     results.append([data_col, "Cohen's d", 'Both','','',d])
    
#     return pd.DataFrame(results[1:],columns=results[0])

# test_assumptions(df_means)

## NORMALIZING ALL LICKS TO AVERAGE OF 4 BASELINE DAYS [NEW 06/29]


In [ ]:
def process_df_to_pivot(df_,phase_dict):
    ## MELTING THE DATA INTO FORM WORKABLE FOR PLOTS/ANALYSIS
    df2 = pd.melt(df_.reset_index(),id_vars=['Mouse_ID','Group','Sex'],
                  value_name='Licks',var_name='Day')
    df2['Phase'] = df2['Day'].map(phase_dict)
    df2['Day_of_Phase'] = df2['Day'].apply(lambda x: x[-1])
    df2 = df2.dropna(subset=['Phase'])
    return df2

In [ ]:
#Calculate average of BL days for each mouse
baseline_lick_avgs = df[BL_cols].mean(axis=1)
baseline_lick_avgs

In [ ]:
df_norm = df.copy()#select_dtypes('number')#.applymap(lambda x:x/baseline_lick_avgs)
## NORMALIZING ALL LICKS TO AVERAGE OF 4 BASELINE DAYS
for col in df_norm.select_dtypes('number').columns:
    df_norm[col] = df_norm[col] / baseline_lick_avgs
df_norm

In [ ]:
df2_norm = process_df_to_pivot(df_norm,phase_dict)
df2_norm